In [ ]:
!pip install detoxify transformers torch flask flask-cors


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [ ]:
from detoxify import Detoxify

text = "you are so good"
result = Detoxify('original').predict(text)
print(result)


{'toxicity': 0.0006655675, 'severe_toxicity': 0.000119363875, 'obscene': 0.00018403796, 'threat': 0.00013163981, 'insult': 0.00017291663, 'identity_attack': 0.00013522622}


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import re
from google.colab import files

uploaded = files.upload()


df = pd.read_csv("labeled_data.csv")


df = df[["tweet", "class"]]
df.columns = ["tweet", "label"]

label_mapping = {0: "neutral", 1: "hate", 2: "offensive"}
df["label"] = df["label"].map(label_mapping)


def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\@\w+|\#", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\b\d+\b","",text)
    text = text.lower().strip()
    return text


df["tweet"] = df["tweet"].apply(clean_text)


print(df.head())
df.to_csv("cleaned_data.csv", index = False)
files.download("cleaned_data.csv")


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Data
df = pd.read_csv("/content/drive/MyDrive/cleaned_data (2).csv")

# Ensure all tweets are strings and remove NaN values
df["tweet"] = df["tweet"].astype(str).str.lower().str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)
df = df.dropna(subset=["tweet"])

# Define Label Mapping for One-Hot Encoding
label_map = {"neutral": 0, "hate": 1, "offensive": 2}

df["label"] = df["label"].map(label_map)
one_hot_labels = torch.nn.functional.one_hot(torch.tensor(df["label"].values), num_classes=3).float()

# Load Model & Tokenizer
MODEL_NAME = "unitary/toxic-bert"
autotokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
automodel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

num_labels = 3
automodel.classifier = nn.Linear(automodel.config.hidden_size, num_labels)
automodel.config.num_labels = num_labels  # Force it to recognize 3 labels

# Initialize Weights
nn.init.xavier_uniform_(automodel.classifier.weight)
automodel.classifier.bias.data.zero_()

# Move Model to GPU
automodel.to(device)

# Tokenize Inputs
encodings = autotokenizer(list(df["tweet"]), truncation=True, padding=True, max_length=128, return_tensors="pt")

# Move Encodings and Labels to GPU
encodings = {key: val.to(device) for key, val in encodings.items()}
one_hot_labels = one_hot_labels.to(device)

# Define Dataset Class
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

dataset = ToxicDataset(encodings, one_hot_labels)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none",
    fp16=True if torch.cuda.is_available() else False  # Enable mixed precision if GPU available
)

# Trainer
trainer = Trainer(
    model=automodel,
    args=training_args,
    train_dataset=dataset
)

# Train the Model
trainer.train()

Using device: cpu


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

import torch
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Initialize Flask app
app = Flask(__name__)

ngrok.set_auth_token("2v31dvvAGAeGU6QOg3FROfPgZOT_69RpPsp8nkj1pSoWdMprL")

# Load model and tokenizer
model_name = "Unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Check model labels
labels = model.config.id2label
print(f"Model Labels: {labels}")

# Prediction function with thresholding
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    print(f"Probabilities: {probabilities.tolist()}")  # ✅ Logs probability values

    max_prob, predicted_class = torch.max(probabilities, dim=-1)

    # Threshold check: If no strong prediction, return "neutral"
    threshold = 0.5
    if max_prob.item() < threshold:
        return "neutral"

    return labels[predicted_class.item()]

# API route for predictions
@app.route('/predict', methods=['POST'])
def predict_api():
    try:
        data = request.get_json(force=True)
        print(f"Received request: {data}")

        if 'text' not in data:
            return jsonify({"error": "Invalid input format"}), 400

        prediction = predict(data['text'])
        response = jsonify({"prediction": prediction})

        print(f"Sending response: {response.get_json()}")
        return response
    except Exception as e:
        print(f"Error: {str(e)}")
        return jsonify({"error": str(e)}), 500

# Start Flask server with ngrok
if __name__ == "__main__":
    public_url = ngrok.connect(5000)  # Only run once
    print(f"Flask app running on: {public_url}")
    app.run(port=5000)


Model Labels: {0: 'toxic', 1: 'severe_toxic', 2: 'obscene', 3: 'threat', 4: 'insult', 5: 'identity_hate'}
Flask app running on: NgrokTunnel: "https://b2c5-34-73-59-103.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received request: {'text': 'you are a nice person'}


INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 16:08:33] "POST /predict HTTP/1.1" 200 -


Probabilities: [[0.48540905117988586, 0.08060965687036514, 0.13110344111919403, 0.0883391723036766, 0.12124878168106079, 0.09328990429639816]]
Sending response: {'prediction': 'neutral'}


...

Received request: {'text': 'you are a liar'}
Probabilities: [[0.8577970266342163, 0.00021083105821162462, 0.005862403195351362, 4.758412978844717e-05, 0.13591498136520386, 0.00016723039152566344]]
Sending response: {'prediction': 'toxic'}


INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 16:11:46] "POST /predict HTTP/1.1" 200 -


Received request: {'text': 'you are a liar'}
Probabilities: [[0.8577970266342163, 0.00021083105821162462, 0.005862403195351362, 4.758412978844717e-05, 0.13591498136520386, 0.00016723039152566344]]
Sending response: {'prediction': 'toxic'}


INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 16:11:55] "POST /predict HTTP/1.1" 200 -


Received request: {'text': 'you are a liar'}
Probabilities: [[0.8577970266342163, 0.00021083105821162462, 0.005862403195351362, 4.758412978844717e-05, 0.13591498136520386, 0.00016723039152566344]]
Sending response: {'prediction': 'toxic'}
